In [79]:
using DifferentialEquations
using ADCME
using PyCall

In [189]:
function compute_cpHS(a, R, T)
    m = size(a)[1]
    cpHS = zeros(m, 3)
    for i = 1:m
        if (T > a[i,1])
            cpHS[i,1] = (a[i,2] + a[i,3] * T + a[i,4] * T^2 + a[i,5] * T^3 + a[i,6] * T^4) * R
            cpHS[i,2] = (a[i,2] + a[i,3] * T / 2 + a[i,4] * T^2 /3 + a[i,5] * T^3 /4 + a[i,6] * T^4 /5 + a[i,7]/T) * R * T
            cpHS[i,3] = (a[i,2] * log(T) + a[i,3] * T + a[i,4] * T^2 /2 + a[i,5] * T^3 /3 + a[i,6] * T^4 / 4 + a[i,8]) * R
        else
            cpHS[i,1] = (a[i,9] + a[i,10] * T + a[i,11] * T^2 + a[i,12] * T^3 + a[i,13] * T^4) * R
            cpHS[i,2] = (a[i,9] + a[i,10] * T / 2 + a[i,11] * T^2 /3 + a[i,12] * T^3 /4 + a[i,13] * T^4 /5 + a[i,14]/T) * R * T
            cpHS[i,3] = (a[i,9] * log(T) + a[i,10] * T + a[i,11] * T^2 /2 + a[i,12] * T^3 /3 + a[i,13] * T^4 / 4 + a[i,15]) * R        
        end
    end
    return cpHS
end

compute_cpHS (generic function with 1 method)

In [190]:
#### Read data
py"""
import numpy as np
"""

In [220]:
V = 1  # Volume
m = 0.21703355349342215 # Total mass
NASA_coeffs = py"np.load"("data/NASA_coeffs.npy")
W = py"np.load"("data/molecular_weights.npy") # Molar weight
ν1_order = py"np.load"("data/reaction_orders.npy") # Forward molar stoichiometric coefficients
ν1 = py"np.load"("data/reactants_stoich_coeffs.npy") # Forward molar stoichiometric coefficients
ν2 = py"np.load"("data/product_stoich_coeffs.npy") # Backward model stoichiometric coefficients
N = size(ν1)[1]  # Number of Species
M = size(ν1)[2]  # Number of Reactions
ν = ν2 - ν1  
pa = 100000 # 1 bar
R = 8314.46261815324 # Gas constant in kmol
### Constants: Combustion Chamber Level
min_dot = 0 # Rate at which mass enters the chamber
mout_dot = 0 # Rate at which mass leaves the chamber
Yin = ones(N) # Mass fraction of species entering the chamber
Yout = ones(N) # Mass fraction of species leaving the chamber
Qdot = 0 # Heating source
m_dot = min_dot - mout_dot
hin = 1 # Enthalpy of input
### Unknowns
T = 1200.0 # Temperature
Y = [0.5; 0; 0.5; 0; 0] # Mass fractions
### Quantities that depend on the Unknowns
ρ = m / V # density
Af = [3.47850e+08] # preexponential constant Afj
β = [0] # Temperature exponent
E = [83680000.0] # Activation energy for the reactions in kJ (divide the value from Cantera by 1000)
Q = ones(M) # Individual progress rates
Kf = ones(M) # Forward reaction coefficients
Kr = zeros(M) # Reverse reaction coefficients
##### Computing ω_dot 
cpHS = compute_cpHS(NASA_coeffs, R, T)
cp = cpHS[:,1]
h = cpHS[:,2]
s = cpHS[:,3]
cvk = cp .- R
ΔS = ν' * s  # Entropy change for reaction j
ΔH = ν' * h # Entahlpy change for reaction j
X = ρ * Y ./ W # Concentration
cv = sum(cvk ./ W .* Y) # Mass heat capacities
u = h ./ W - R ./ W .* T   # Internal energy for species
p = sum(X) * R * T # pressure
Kf = Af .* (T .^ β) .* exp.(-E ./ (R * T))
Kr = Kf ./ (((pa/(R * T)) .^ sum(ν, dims=1)' .* exp.(ΔS ./ R - ΔH ./ (R * T))))
Q = Kf .* prod(X .^ ν1_order, dims=1)' .- Kr .* prod(X .^ ν2, dims=1)'
ω_dot = W .* sum(ν .* Q', dims=2)
###### Species Conservation
mgen_dot = V .* ω_dot
Y_dot = (1 / m) .* (min_dot .* (Yin .- Y) - mout_dot .* Y .+ mgen_dot) 
###### Energy Conservation
T_dot = 1 / (m * cv) * (-Qdot + min_dot * (hin - sum(u .* Yin)) - p * V / m * mout_dot - sum(mgen_dot .* u))

4.268052226065232e7

In [305]:
function f(TY)
    T = TY[1]
    Y = TY[2:end]
    cpHS = compute_cpHS(NASA_coeffs, R, T)
    cp = cpHS[:,1]
    h = cpHS[:,2]
    s = cpHS[:,3]
    cvk = cp .- R
    ΔS = ν' * s  # Entropy change for reaction j
    ΔH = ν' * h # Entahlpy change for reaction j
    X = ρ * Y ./ W # Concentration
    cv = sum(cvk ./ W .* Y) # Mass heat capacities
    u = h ./ W - R ./ W .* T   # Internal energy for species
    p = sum(X) * R * T # pressure
    Kf = Af .* (T .^ β) .* exp.(-E ./ (R * T))
    Kr = Kf ./ (((pa/(R * T)) .^ sum(ν, dims=1)' .* exp.(ΔS ./ R - ΔH ./ (R * T))))
    Q = Kf .* prod(max.(X,0) .^ ν1_order, dims=1)' .- Kr .* prod(X .^ ν2, dims=1)'
    ω_dot = W .* sum(ν .* Q', dims=2)
    ###### Species Conservation
    mgen_dot = V .* ω_dot 
    Y_dot = (1 / m) .* (min_dot .* (Yin .- Y) - mout_dot .* Y .+ mgen_dot) 
    ###### Energy Conservation
    T_dot = 1 / (m * cv) * (-Qdot + min_dot * (hin - sum(u .* Yin)) - p * V / m * mout_dot - sum(mgen_dot .* u))
    return [T_dot; Y_dot]
end

f (generic function with 4 methods)

In [321]:
## RK4
dt = 1e-9
n_t = 10000
T = 1200.0 # Temperature
Y = [0.5; 0; 0.5; 0; 0] # Mass fractions
TY = [T;Y]
Tt = zeros(n_t)
Pt = zeros(n_t)
Yt = zeros(N,n_t)
Y0 = [0.5; 0; 0.5; 0; 0] 
for i = 1:n_t
    k1 = f(TY)
    k2 = f(TY + 0.5 * dt * k1)
    k3 = f(TY + 0.5 * dt * k2)
    k4 = f(TY + dt * k3)
    TY += 1/6 * dt * (k1 + 2 * k2 + 2 * k3 + k4)
    Tt[i] = TY[1]
    Y = TY[2:end]
    Yt[:,i] = Y
    X = ρ * Y ./ W
    p = sum(X) * R * TY[1]
    Pt[i] = p
end


In [322]:
X = [i * 1e-9 for i = 1:n_t]
Yt_cant = py"np.load"("data/mass_fraction.npy")
plot(X, Yt', label=["O2" "H20" "CH4" "CO2" "N2"])
xlabel!("Time")
ylabel!("Mass Fractions")
savefig("Mass_fraction.pdf")

Pt_cant = py"np.load"("data/pressure.npy")
plot(X, [Pt Pt_cant], label = ["Julia" "Cantera"])
xlabel!("Time")
ylabel!("Pressure")
savefig("Pressure.pdf")

plot(X, Tt)
Tt_cant = py"np.load"("data/temperature.npy")
plot(X, [Tt Tt_cant], label = ["Julia" "Cantera"])
xlabel!("Time")
ylabel!("Temperature")
savefig("Temperautre.pdf")